# Анализ пользователей Яндекс.Музыки в Москве и Санкт-Петербурге

## 1. Обзор данных

In [1]:
import pandas as pd

### 1.1 Функции <a id="1.1"></a>

#### 1.1.1 Функция для первичного анализа <a id="1.1.1"></a>

In [2]:
def first_rev(df):
    print('info')
    display(df.info(memory_usage='deep'))
    print('describe')
    display(df.describe().T.round(3))
    print('head')
    display(df.head())
    print('duplicated')
    display(df.duplicated().sum())

#### 1.1.2 Функция для быстрой замены неявных дубликатов <a id="1.1.2"></a>

In [3]:
def replace_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre)

#### 1.1.3 Функция для оценки информации о самых популярных жанрах в определенный день и время <a id="1.1.3"></a>

In [4]:
# функция с 4 параметрами: датафрейм, день недели, время начала, время окончания,
# которая возвращает информацию о самых популярных жанрах
# в указанный день в заданное время
def genre_weekday(df_city, day, time1, time2):
    genre_df = df_city[
        (df_city['day'] == day)
        & (df_city['time'] > time1)
        & (df_city['time'] < time2)
    ]
    # количество записей для каждого из присутствующих жанров
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    # сортируем по убыванию встречаемости
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    # возвращаем топ-10 популярных жанров (в указанный день, в заданное время)
    return(genre_df_sorted.head(10))

### 1.2 Данные <a id="1.2"></a>

In [5]:
df = pd.read_csv(r"C:\Users\csvic\Desktop\data analysis\Проекты для git\1. Яндекс.Музыка.csv")

In [6]:
first_rev(df)

info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 29.1 MB


None

describe


,count,unique,top,freq
userID,65079,41748,A8AE9169,76
Track,63848,47245,Intro,34
artist,57876,43605,Sasha,6
genre,63881,289,pop,8850
City,65079,2,Moscow,45360
time,65079,20392,08:14:07,14
Day,65079,3,Friday,23149


head


,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


duplicated


3826

### 1.3 Вывод

Согласно документации к данным:
* `userID` — идентификатор пользователя
* `Track` — название трека
* `artist` — имя исполнителя
* `genre` — название жанра
* `City` — город пользователя
* `time` — время начала прослушивания
* `Day` — день недели

**Дополнительно:**
* Изменить названия колонок для удобства анализа.
* Удалить дубликаты.
* Проверить и заменить/удалить пропуски значений в колонках `Track`, `artist` и `genre`.
* Оптимизировать тип данных.
* Проверить колонку `genre` на неявные дубликаты.

## 2. Предобработка данных

### 2.1 Стиль заголовков <a id="2.1"></a>
Изменим стиль заголовков.

In [7]:
df.columns = [x.lower().replace(' ', '') for x in df.columns]

In [8]:
df.columns

Index(['userid', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### 2.2 Дубликаты <a id="2.2"></a>
Удалим явные дубликаты.

In [9]:
df = df.drop_duplicates().reset_index(drop=True)

In [10]:
df.duplicated().sum()

0

### 2.3 Пропуски значений <a id="2.3"></a>

* Заменим на `'unknown'` пропуски значений в `track` и `artist`, так как они не важны в данном исследовании.
* В идеальном случае надо выявить причину пропусков в `genre`, хоть и доля пропусков всего 2%. В учебном проекте возможности узнать причину пропусков нет, поэтому их также заменим на `'unknown'`, чтобы позже можно было оценить влияние пропусков на ход исследования.

In [11]:
df.isna().sum()

userid       0
track     1155
artist    6772
genre     1127
city         0
time         0
day          0
dtype: int64

In [12]:
df.isna().mean().round(2)

userid    0.00
track     0.02
artist    0.11
genre     0.02
city      0.00
time      0.00
day       0.00
dtype: float64

In [13]:
columns_to_replace = ['track', 'artist', 'genre']
for colomns in columns_to_replace:
    df[colomns] = df[colomns].fillna('unknown')

In [14]:
df.isna().sum()

userid    0
track     0
artist    0
genre     0
city      0
time      0
day       0
dtype: int64

### 2.4 Тип данных <a id="2.4"></a>
Изменим тип данных для удобства и скорости анализа.

In [15]:
# не меняем тип данных у колонки `time`, так как используем это значение в функции
for column in df.columns:
    if column != 'time':
        df[column] = df[column].astype('category')

In [16]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61253 entries, 0 to 61252
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   userid  61253 non-null  category
 1   track   61253 non-null  category
 2   artist  61253 non-null  category
 3   genre   61253 non-null  category
 4   city    61253 non-null  category
 5   time    61253 non-null  object  
 6   day     61253 non-null  category
dtypes: category(6), object(1)
memory usage: 17.4 MB


#### 2.4.1 Вывод
Изменили тип данных и уменьшили размер используемой памяти с 27.7 MB до 15.2 MB.

### 2.5 Проверка на нормальность <a id="2.5"></a>
Проверим неявные дубликаты и выбивающиеся значения в колонке `genre`.

In [17]:
df['genre'].value_counts()

pop                 8323
dance               6367
rock                5844
electronic          5522
hip                 2975
                    ... 
malaysian              1
lovers                 1
loungeelectronic       1
laiko                  1
электроника            1
Name: genre, Length: 290, dtype: int64

In [18]:
sorted(df['genre'].unique())

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'alternativepunk',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'author',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'chanson',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',


In [19]:
replace_genres(['hip', 'hop', 'hip-hop'], 'hiphop')
replace_genres(['электроника'], 'electronic')
replace_genres(['ïîï'], 'unknown')

In [20]:
sorted(df['genre'].unique())

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'alternativepunk',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'author',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'chanson',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',


#### 2.5.1 Вывод
Удалили неявные дубликаты в `genre`.

## 3. Проверка гипотез

### 3.1 Активность пользователей в Москве и Санкт-Петербурге зависит от дня недели и проявляется по-разному <a id="3.1"></a>
Проверяем гипотезу по данным о трех днях недели — понедельник, среда и пятница.

In [21]:
h1_week = df.pivot_table(index='city', values='track', aggfunc='count')
h1_week['perc'] = (h1_week['track'] / h1_week['track'].sum() * 100).round(1)
h1_week

,track,perc
city,,
Moscow,42741,69.8
Saint-Petersburg,18512,30.2


In [22]:
h1_days = df.pivot_table(index='city', columns='day', values='track', aggfunc='count').T
h1_days['perc_msc'] = (h1_days['Moscow'] / h1_days['Moscow'].sum() * 100).round(1)
h1_days['perc_spb'] = (h1_days['Saint-Petersburg'] / h1_days['Saint-Petersburg'].sum() * 100).round(1)
h1_days

city,Moscow,Saint-Petersburg,perc_msc,perc_spb
day,,,,
Friday,15945,5895,37.3,31.8
Monday,15740,5614,36.8,30.3
Wednesday,11056,7003,25.9,37.8


#### 3.1.1 Вывод
* В выборке 70% данных о слушателях в Москве и 30% — о слушателях в Санкт-Петербурге.

**Выявили, что активность пользователей в Москве и Санкт-Петербурге зависит от дня недели и проявляется по-разному:**
* В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
* В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

### 3.2 Преобладание жанров в понедельник и пятницу в разное время дня в Москве и Санкт-Петербурге различается <a id="3.2"></a>
В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [23]:
moscow = df.query('city == "Moscow"')
spb = df.query('city == "Saint-Petersburg"')

In [24]:
genre_weekday(moscow, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [25]:
genre_weekday(spb, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [26]:
genre_weekday(moscow, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [27]:
genre_weekday(spb, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

#### 3.2.1 Вывод

**Сравнение топ-10 жанров в понедельник утром:**

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошел жанр “world”, а в петербургский — джаз и классика.
2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

**Сравнение топ-10 жанров в пятницу вечером:**

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остается тем же самым.

<br>Таким образом, **вторая гипотеза подтвердилась лишь частично**:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные данные о жанрах.

### 3.3 Пользователи Яндекс.Музыки в Москве и Санкт-Петербурге предпочитают разные жанры музыки <a id="3.3"></a>
В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

In [28]:
moscow.groupby('genre')['genre'].count().sort_values(ascending=False).head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [29]:
spb.groupby('genre')['genre'].count().sort_values(ascending=False).head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

#### 3.3.1 Вывод

**Гипотеза частично подтвердилась:**
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 

## 4. Итоги исследования <a id="4"></a>

Проверили три гипотезы и установили:

**Первая гипотеза полностью подтвердилась.** День недели по-разному влияет на активность пользователей в Москве и Петербурге. 


**Вторая гипотеза подтвердилась лишь отчасти.** Этот результат мог оказаться иным, если бы не пропуски в данных.

Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра *'world'*,
* в Петербурге — джаз и классику.


**Третья гипотеза не подтвердилась.** Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.